<a href="https://colab.research.google.com/github/diegohugo570/backup-python/blob/main/01_LLM_Routing_DascIA_Academy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Routing
Fala, pessoal! Vamos começar mais um material prático aqui da Formação Arquiteto de IA.

E nesse material vamos explorar as principais técnicas de LLM Routing modernas, abrangindo as principais técnicas:
- Router preditivo
- Router não-preditivo
- Router híbrido

As características principais dessa biblioteca são:
- **Substituição direta do cliente da OpenAI** (ou execução de um servidor compatível com OpenAI) para rotear consultas mais simples para modelos mais baratos.
- **Roteadores treinados são fornecidos prontos para uso**, e já demonstramos que eles reduzem os custos em até 85%, mantendo 95% da performance do GPT-4 em benchmarks amplamente utilizados como o MT Bench.
- Os benchmarks também mostram que esses roteadores atingem a mesma performance de soluções comerciais, **sendo mais de 40% mais baratos**.
- **O framework pode ser facilmente estendido para incluir novos roteadores** e comparar a performance entre eles em diversos benchmarks.

Então, vamos começar instalando as bibliotecas necessárias

## Router Preditivo

In [ ]:
# Instalando o routellm e o gradio
!pip install -qU "routellm[serve,eval]" gradio groq openai --upgrade

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━

### Configuração Inicial
Antes de continuarmos, é importante mencionar que vamos usar de referência para escolher os modelos os [provedores que tem suporte](https://docs.litellm.ai/docs/providers) pelo LiteLLM.

No caso, vou usar o GPT-4o, da OpenAI, o llama3-70b, da Meta, mas utilizado através dos serviços da Groq.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

### Escolha dos modelos
Aqui, vamos utilizar um router preditivo para escolher entre dois modelos:
- Modelo forte (*strong model*)
- Modelo fraco (*weak model*)

Além disso, vamos usar o *Matrix Factorization router* para fazer o roteamento. O RouteLLM possui as seguintes opções para routing:
-  ***mf***: Usa um modelo de fatorização matricial treinado nos dados de preferência (recomendado)
-  ***sw_ranking***: Use um cálculo Elo ponderado para fazer o *routing*, onde cada voto é ponderado de acordo com quão similar é em relação ao prompt do usuário
-  ***bert***: Usa o classificador BERT treinado nos dados de preferência
-  ***causal_llm***: Usa um classificador baseado em LLM 'fine-tunado' nos dados de preferência
-  ***random***: Faz o roteamento aleatório

Na maioria dos casos, o uso do MF é recomendado, por ser muito bom e leve.

In [ ]:
from routellm.controller import Controller
client = Controller(
  routers=["mf"],
  strong_model="gpt-4o",
  weak_model="groq/llama3-8b-8192",
)

model.safetensors:   0%|          | 0.00/820k [00:00<?, ?B/s]

**Obs.**: Mesmo que não utilizemos algum modelo da OpenAI, ainda assim é necessário a chave api dela, porque é utilizado para gerar os embeddings dos routers ***mf*** e ***sw_ranking***.

### Definindo limites de custo
Cada requisição de roteamento possui um limite de custo que controla o equilíbrio entre custo e qualidade. Devemos calibrar esse limite com base nos tipos de consultas que recebemos para maximizar a performance do roteamento.

Como exemplo, vamos calibrar nosso limite para que 50% das chamadas sejam feitas com o GPT-4o, utilizando dados do Chatbot Arena.

In [ ]:
!python -m routellm.calibrate_threshold --routers mf --strong-model-pct 0.5

README.md: 100% 418/418 [00:00<00:00, 2.62MB/s]
train-00000-of-00001.parquet: 100% 2.11M/2.11M [00:00<00:00, 40.7MB/s]
Generating train split: 100% 57477/57477 [00:00<00:00, 870926.76 examples/s]
For 50.0% strong model calls for mf, threshold = 0.11593


**Obs.**: Isso significa que queremos usar 0,11593 como nosso threshold, de modo que aproximadamente 50% de todas as consultas (aquelas que mais exigem o GPT-4o) sejam roteadas para ele.

#### Sobre os Thresholds
O threshold usado no roteamento controla o equilíbrio entre custo e qualidade. A faixa de valores significativos de threshold varia dependendo do tipo de roteador e das consultas recebidas. Por isso, **recomendo calibrar os thresholds usando uma amostra das suas consultas reais**, além de definir a porcentagem de consultas que você deseja rotear para o modelo mais forte.

Por padrão, o RouteLLM oferece suporte à calibração de thresholds com base no dataset público Chatbot Arena. No entanto, observe que como os thresholds são calibrados com base em um dataset existente, a porcentagem real de chamadas roteadas para cada modelo pode variar de acordo com as consultas reais recebidas.

Por isso, **recomendo calibrar usando um dataset que se aproxime ao máximo das consultas que você realmente processa**.

### Testando o Router
Agora, vamos atualizar o campo model ao gerar as respostas para especificar o roteador e o threshold que desejamos usar:

In [ ]:
response = client.chat.completions.create(
  # router-[ROUTER]-[LIMIAT]
  # Isso diz ao RouteLLM para usar o MF router com um threshold de custo de 0.11593
  model="router-mf-0.11593",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

response

ModelResponse(id='chatcmpl-af4b36b3-620e-4ae2-ad12-61891621a1c3', created=1746387407, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_dadc9d6142', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=26, prompt_tokens=12, total_tokens=38, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.019057831, prompt_time=0.001815477, completion_time=0.021666667, total_time=0.023482144), usage_breakdown={'models': None}, x_groq={'id': 'req_01jtect4emenfr4ry7dv311jzw'})

E pronto! Agora, as requisições serão roteadas entre o modelo forte e o modelo fraco conforme a necessidade, **reduzindo custos enquanto mantém uma alta qualidade nas respostas**.

Dependendo do seu caso de uso, pode ser interessante:
- usar um par de modelos diferente;
- modificar a configuração;
- ou calibrar os thresholds com base nos tipos de consultas que você realmente recebe — tudo para otimizar a performance do roteador.

### Inicializando um servidor
O RouteLLM oferece um servidor leve e compatível com OpenAI para rotear requisições com base em diferentes estratégias de roteamento.

- ***--routers*** define a lista de roteadores disponíveis para o servidor. No exemplo abaixo, o servidor é iniciado com um roteador disponível: mf
- ***--config*** define o caminho para o arquivo de configuração dos roteadores. Se não for especificado, o servidor usará, por padrão, a configuração de melhor desempenho fornecida.

Ao fazer uma requisição para o servidor, o cliente deve especificar o roteador e o threshold de custo a ser usado para cada requisição no campo model, com o seguinte formato:

```router-[NOME_DO_ROUTER]-[THRESHOLD]```

Por exemplo:

```modelo = router-mf-0.5```

Isso indica que a requisição deve ser roteada usando o roteador mf com um threshold de 0.5.

In [ ]:
!nohup python -m routellm.openai_server --routers mf --strong-model gpt-4o --weak-model groq/llama3-70b-8192 > server.log 2>&1 &

In [ ]:
!python -m examples.router_chat --router mf --threshold 0.11593

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
* Running on local URL:  http://127.0.0.1:8001
* Running on public URL: https://cfa933a6058b24140f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
ChatCompletionChunk(id='chatcmpl-d734daa8-90e0-4ab9-9ce6-b3a7ae0f88c8', choices=[Choice(delta=ChoiceDelta(content='Hello', function_call=None, refusal=None, role='assistant', tool_calls=None, provider_specific_fields=None, audio=None), finish_reason=None, index=0, logprobs=None)], created=1746386073, model='llama3-70b-8192', object='chat.completion.chunk', service_tier=None, 

## Roteamento Não-Preditivo


### Execução Paralela

Este notebook demonstra dois tipos de roteamento não-preditivo:

1. **Cascata (Stage-up):** sobe progressivamente entre modelos baratos → intermediários → caros, apenas se necessário.
2. **Execução Paralela:** todos os modelos geram respostas ao mesmo tempo, e um LLM escolhe a melhor.

Usaremos o modelo `llama3-70b-8192` via Groq como julgador.

In [ ]:
from groq import Groq
from openai import ChatCompletion

client_groq = Groq(api_key = userdata.get("GROQ_API_KEY"))

In [ ]:
def julgar_resposta(modelo, resposta, pergunta):
    prompt = f"""
    Você é um avaliador de qualidade de respostas geradas por IA.
    Sua tarefa é dizer se a resposta abaixo é suficiente para a pergunta, com base em completude, exatidão e clareza.

    Pergunta: {pergunta}

    Resposta do modelo {modelo}: {resposta}

    Responda apenas com "ACEITA" ou "REJEITA", sem explicações.
    """

    resposta = client_groq.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return resposta.choices[0].message.content.strip()


### Roteamento em Cascata

In [ ]:
pergunta = "A que temperatura a água ferve ao nível do mar?"
resposta_mistral = "A água ferve em olá, olá!!!"

if julgar_resposta("mistral", resposta_mistral, pergunta).strip().upper() == "ACEITA":
    print("✅ Mistral aceito")
else:
    print("Mistral não aceito.")
    resposta_deepseek = "Ao nível do mar, a água entra em ebulição a 100 graus Celsius."
    if julgar_resposta("deepseek", resposta_deepseek, pergunta).strip().upper() == "ACEITA":
        print("✅ DeepSeek aceito")
    else:
        resposta_gpt4 = "A água entra em ebulição exatamente a 100°C ao nível do mar, sob pressão atmosférica padrão."
        print("✅ GPT-4 enviado")

Mistral não aceito.
✅ DeepSeek aceito


### Julgamento Paralelo

In [ ]:
def escolher_melhor_resposta(modelos_e_respostas, pergunta):
    comparativo = "\n\n".join(
        [f"Resposta de {modelo}:\n{resposta}" for modelo, resposta in modelos_e_respostas.items()]
    )

    prompt = f"""
    Você é um avaliador de qualidade de respostas geradas por IA.
    Sua tarefa é analisar as respostas abaixo para a mesma pergunta e indicar **qual delas é a melhor** com base em completude, exatidão e clareza.

    Pergunta: {pergunta}

    {comparativo}

    Responda apenas com o nome do modelo. Sem explicações.
    """

    response = client_groq.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [ ]:
respostas = {
    "mistral": "A água ferve em 100 graus.",
    "deepseek": "Ao nível do mar, a água entra em ebulição a 100 graus Celsius.",
    "gpt-4": "A água entra em ebulição exatamente a 100°C ao nível do mar, sob pressão atmosférica padrão."
}

modelo_escolhido = escolher_melhor_resposta(respostas, pergunta)
print(f"🎯 Melhor resposta: {modelo_escolhido}")
print("Resposta retornada:", respostas[modelo_escolhido])

🎯 Melhor resposta: gpt-4
Resposta retornada: A água entra em ebulição exatamente a 100°C ao nível do mar, sob pressão atmosférica padrão.


## Router Híbrido

Este notebook executa uma abordagem híbrida realista:

1. Usa `RouteLLM` com o roteador `mf` para prever se o modelo leve pode ser usado.
2. Se for possível, executa o modelo leve (`llama3-70b` da Groq).
3. A resposta é julgada com um LLM (o próprio LLaMA3 via Groq).
4. Se a resposta for rejeitada, sobe para `GPT-4o`.

Economiza quando possível, garante qualidade quando necessário.

In [ ]:
controller = Controller(
    routers=["mf"],
    strong_model="gpt-4o",
    weak_model="groq/llama3-70b-8192",
)

# Setup Groq client para usar como julgador
judge = Groq(api_key="SUA_GROQ_API_KEY")

In [ ]:
import requests

def chamar_openai(pergunta):
    api_key = os.getenv("OPENAI_API_KEY")
    url = "https://api.openai.com/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": "gpt-4o",
        "messages": [
            {"role": "developer", "content": "You are a helpful assistant that answers in pt-BR."},
            {"role": "user", "content": pergunta}
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()  # lança erro se falhar

    return response.json()["choices"][0]["message"]["content"]

In [ ]:
pergunta = "how much is 484ˆ4 divided by 38?"

# Etapa 1: previsão com o roteador (sem execução ainda)
response = controller.chat.completions.create(
      model="router-mf-0.11593",
      messages=[{"role": "user", "content": pergunta}])

roteado_para = response.model

if roteado_para == "llama3-70b-8192":
    print("🔄 RouteLLM indicou uso do modelo leve (Groq LLaMA3).")

    # Executa modelo leve
    resposta_leve = response.choices[0].message.content

    # Julga qualidade
    veredito = julgar_resposta("llama3", resposta_leve, pergunta)

    if veredito == "ACEITA":
        print("✅ Resposta do modelo leve foi aceita.")
        print(resposta_leve)
    else:
        print("🔁 Subindo para GPT-4o...")
        resposta_forte = chamar_openai(pergunta)
        print("✅ Resposta final do GPT-4o:")
        print(resposta_forte)

else:
    print("🔼 RouteLLM já recomendou direto o modelo forte (GPT-4o).")
    resposta_forte = response.choices[0].message.content
    print("✅ Resposta do GPT-4o:")
    print(resposta_forte)

🔼 RouteLLM já recomendou direto o modelo forte (GPT-4o).
✅ Resposta do GPT-4o:
To find the value of \(484^4\) divided by 38, we need to first calculate \(484^4\) and then divide the result by 38.

1. Calculate \(484^4\):

   \[
   484^4 = (484 \times 484) \times (484 \times 484)
   \]

   First, calculate \(484 \times 484\):

   \[
   484 \times 484 = 234,256
   \]

   Then, multiply the result by itself to find \(484^4\):

   \[
   234,256^2 = 54,595,979,776
   \]

2. Divide the result by 38:

   \[
   \frac{54,595,979,776}{38} \approx 1,436,736,309.368
   \]

So, \(484^4\) divided by 38 is approximately 1,436,736,309.368.
